**Agrupamento - Mineração de Dados**


**Nome: Davi Augusto Neves Leite**

**Data de Entrega: 31/10/2023**


---


# **Materiais**


Os principais recursos para a execução desta atividade podem ser vistos a seguir.

1. **Software**

- Sistemas Operacionais: Windows 11 para _desktop_;
- Ambiente de Desenvolvimento Integrado: Microsoft Visual Studio Code;
- Linguagem de Programação: Python 3.12.0 64-bit.

2. **Hardware**

- Notebook pessoal Lenovo Ideapad 330-15IKB com: processador Intel Core i7-8550U, HDD WD Blue WD10SPZX de 1TB, SSD Crucial BX500 de 1TB, 12 GB DDR4 de Memória RAM e placa de vídeo NVIDIA GeForce MX150 (2 GB GDDR5 de memória).


---


# **Instalação das Bibliotecas Principais**

Nota: ao decorrer deste Notebook, outras bibliotecas podem ser utilizadas em quaisquer respectiva seção/conjunto de dados, dependendo da necessidade. Abaixo, há a instalação das principais que são comuns e utilizadas em todas ou quase todas seções/conjunto de dados.


In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install plotly
%pip install scikit-learn


---


# **Importação das Bibliotecas Principais**

Nota: ao decorrer deste Notebook, outras bibliotecas podem ser utilizadas em quaisquer respectiva seção/conjunto de dados, dependendo da necessidade. Abaixo, há a importação das principais que são comuns e utilizadas em todas ou quase todas seções/conjunto de dados.


In [ ]:
import numpy as np  # Manipulação de listas
import pandas as pd  # Manipulação de tabelas
import seaborn as sbn  # Geração de gráficos estatísticos
import plotly.express as px  # Outro para geração de gráficos
import matplotlib.pyplot as plt  # Geração de gráficos de listas
import sklearn as skl  # Biblioteca para pré-processamento
from copy import copy as cp  # Possibilitar copiar os objetos

# Ignorar os avisos não importantes durante a execução deste notebook
import warnings

warnings.filterwarnings("ignore")

---


# **Conjunto de Dados: _Fashion MNIST_**


**Descrição do Dataset:** este conjunto é composto por **70 mil imagens** a respeito de **10 peças de roupas distintas**. Cada imagem possui a **resolução de 28x28 (784 pixels) em escala de cinza e 256 níveis de cinza**. Desse total, exitem **7000 imagens** para cada uma das peças de roupas, ou classes.

Cada classe pode ser vista a seguir.

1. Camiseta
2. Calça
3. Pulôver
4. Vestido
5. Casaco
6. Sandália
7. Camisa
8. Tênis
9. Bolsa
10. Bota de Tornozelo

Este conjunto de dados pode ser acessado por meio de: [Fashion MNIST](https://www.openml.org/search?type=data&status=active&id=40996)
(última data de acesso: 18 de out. de 2023).


- **Para fins práticos, apenas as classes _Camiseta_, _Vestido_, _Casaco_ e _Camisa_ serão utilizadas para esta atividade, tendo em vista que, por se tratar de uma base de dados de imagens, o processamento necessário para classificação é elevado.**


## **Importação da Base de Dados**


Para importar as imagens, utiliza-se o módulo **scikit-learn** capaz de carregar diversas bases de dados através do portal **OpenML**, o qual essa base de dados está disponível online.


In [ ]:
# Importação da função necessária para importação de base de dados OpenML
from sklearn.datasets import fetch_openml

# Carrega as imagens e suas classes separadamente
images, targets = fetch_openml(
    "Fashion-MNIST", return_X_y=True, as_frame=True, parser="auto"
)

# Conversão das imagens para NumPy
images = images.to_numpy()
targets = targets.to_numpy().astype(int)

## **Pré-Processamento dos Dados**


### \* Separação das Classes de Interesse

Inicia-se com a separação das classes de interesse, isto é: _Camiseta_, _Vestido_, _Casaco_ e _Camisa_. Cada qual será referida, respectivamente, pela identificação de 0 a 3 (_0 - Camiseta_, _1 - Vestido_, ...).


In [ ]:
# Definição das classes de interesse
labels_class = {0: 0, 3: 1, 4: 2, 6: 3}

# Variáveis de seleção das imagens e classes respectivas
selected_images = []
selected_targets = []

# Criação de um dicionário com os rótulos das classes para fácil acesso
labels_desc = {0: "camiseta", 1: "vestido", 2: "casaco", 3: "camisa"}
labels_names = ["Camiseta", "Vestido", "Casaco", "Camisa"]

# Percorre as imagens da base de dados original
for idx, label in enumerate(targets):
    # Para cada rótulo de interesse, salvar as imagens e os rótulos
    if label in labels_class.keys():
        selected_images.append(images[idx])
        selected_targets.append(labels_class[targets[idx]])

# Conversão para NumPy
selected_images = np.array(selected_images)
selected_targets = np.array(selected_targets)

print("Imagens Selecionadas:\n", selected_images)
print("\nRótulos Selecionados: ", selected_targets)
print("\nTotal de Dados: ", selected_images.shape[0])
print("\nCaracterísticas Totais: ", selected_images.shape[1])

### Exibição das Classes de Interesse

Abaixo, é possível visualizar uma amostra de cada classe, por meio do **matplotlib**.


In [ ]:
# Selecionando um índice de cada classe
idx_0 = np.where(selected_targets == 0)[0][0]
idx_1 = np.where(selected_targets == 1)[0][0]
idx_2 = np.where(selected_targets == 2)[0][0]
idx_3 = np.where(selected_targets == 3)[0][0]
idx_example_images = np.array([idx_0, idx_1, idx_2, idx_3])

# Definindo o tamanho da figura
plt.figure(figsize=(10, 8))

# Definindo o número de linhas e colunas das subfiguras
fig_n_rows = 1
fig_n_cols = 5

# Mostrando as amostras de cada classe
for label, image_idx in enumerate(labels_desc.keys()):
    plt.subplot(fig_n_rows, fig_n_cols, label + 1)
    plt.title(f'Classe "{labels_desc[label]}"')
    plt.imshow(
        selected_images[idx_example_images[image_idx]].reshape(28, 28), cmap="gray"
    )
    plt.axis("off")
plt.show()

### Tratamento de Dados Perdidos ou Inexistentes (NaN)


Para verificar se algum dado está faltando, **caso não seja indicado pela descrição do _dataset_**, pode ser realizado a seguinte operação de força-bruta:


In [ ]:
# Verificando o número de dados faltantes a partir do NumPy
missing_selected_image = np.isnan(selected_images)
missing_selected_image = np.sum(missing_selected_image)
print("Número de Dados Perdidos: {0}".format(missing_selected_image))

Como é possível ver, não há nenhum dado perdido neste _dataset_ e, desta forma, não é necessário realizar nenhum método de tratamento neste contexto.


### Normalização dos Dados


Para normalizar os dados via _Standardization (Z-Score)_ deste **dataset**, basta aplicar as seguintes operações:


In [ ]:
# Função responsável pela normalização via Z-Score
from sklearn.preprocessing import StandardScaler

# Mostrando os dados não normalizados
print("Dados Não Normalizados")
print(
    "\tMédia: {0} | Desvio-Padrão: {1}".format(
        np.mean(selected_images), np.std(selected_images)
    )
)
print(selected_images)


# Aplicando a Normalização com Z-Score
selected_images = StandardScaler().fit_transform(selected_images)
print("\n")

# Mostrando os dados normalizados
print("Dados Normalizados com Z-Score")
print(
    "\tMédia: {0} | Desvio-Padrão: {1}".format(
        np.mean(selected_images), np.std(selected_images)
    )
)
print(selected_images)

### \* Redução de Dimensionalidade: _Principal Component Analysis_


Para reduzir a dimensionalidade deste **dataset**, é recomendado o uso do _Principal Component Analysis_ (PCA). Desta forma:


In [ ]:
# Importação da função do PCA do sklearn
from sklearn.decomposition import PCA

# Definindo o número de componentes do PCA
n_components = 0.9

# Aplicando o PCA
pca = PCA(n_components=n_components, copy=True, whiten=False)
projected_data = pca.fit_transform(selected_images)

# Mostrando os dados projetados com PCA
print("Dados Projetados com PCA")
print(projected_data)
print("\n")

# Segundo: mostrando a matriz de covariância do PCA
print("Variâncias")
print(pca.explained_variance_ratio_)
print("\n")

# Transformando o conjunto de dados em DataFrame para melhor manipulação
column_name = ["pixel" + str(i) for i in range(1, 785)]
selected_images_zscore_frame = pd.DataFrame(selected_images, columns=column_name)

# Terceiro: mostrando os componentes do PCA
component_names = ["component {}".format(i) for i in range(len(pca.components_))]
components_pca = pd.DataFrame(
    data=pca.components_,
    index=component_names,
    columns=selected_images_zscore_frame.columns,
)
components_pca.head()

É possível, por exemplo, visualizar a variação do PCA na medida em que se aumentam as características (ou dimensões). A chamada **variância explicada** exprime exatamente a ideia de quanta variabilidade é possível capturar do conjunto de dados, sendo possível, a partir desta análise, levar a um melhor desempenho de treinamento do modelo.


In [ ]:
import plotly.graph_objects as px_go

# Gráfico da variância do PCA, a partir dos componentes
# exp_var_cum = np.cumsum(pca.explained_variance_ratio_)
exp_var_cum = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3) * 100)

fig_pca = px_go.Figure(
    data=px_go.Scatter(x=list(range(1, len(exp_var_cum) + 1)), y=exp_var_cum)
)
fig_pca.update_layout(
    title="Variância Explicada do PCA",
    xaxis_title="# de Características",
    yaxis_title="% Variância Explicada",
)

---


## **Agrupamento**


Texto


### Subtítulo


Texto


---
